In [58]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler


# from DataProduction import dataProd

import warnings
warnings.filterwarnings('ignore')

# Initialize scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Data production and loading
run_list = [285490, 286498, 287987, 288543, 289413, 289733, 291970, 293528]
planes_list = ['UTaX', 'UTaU', 'UTbV', 'UTbX']

# dataProduction = dataProd()
# mix_ = dataProduction.multipleRun(*run_list)
# mix = mix_.copy()

import random
random.seed(46)

from scipy.special import kl_div,softmax

sns.set(
    context='notebook',  # 'paper', 'notebook', 'talk', 'poster'
    style='darkgrid',   # 'darkgrid', 'whitegrid', 'dark', 'white', 'ticks'
    palette='deep',      # Color palette
    font='sans-serif',   # Font family
    font_scale=0.75,      # Font scale
    rc={'figure.figsize': (12, 6)}  # Default figure size
)


def colorCodes(div_value):

    if div_value < 0.1:
        color = 'green'
    elif (div_value > 0.1) & (div_value<0.6):
        color = 'orange'
    else:
        color = 'red'
    return f'background-color: {color}'

### Simulation Data Generation.

In [59]:
class simData():

    def __init__(self,num_chips,pseudo_random):

        np.random.seed(pseudo_random)
        self.num_chips = num_chips

        chips = {}
        for i in range(self.num_chips): #One side of 1024 chips [512]
            chips[f'chip{i+1}'] = np.random.laplace(loc=0.0, scale=1.0, size=128) # For 1 Plane, 1024 ASICS, 128 Channel Each

        self.all_chips_pedestals = pd.DataFrame(chips)

        # return self.all_chips_pedestals

    def pedestals_data(self):
        return self.all_chips_pedestals

    def chip_average(self):
        self.average_pedestal_per_chip = np.mean(self.all_chips_pedestals,axis=0).values#.reshape(int(self.num_chips/16),16)

        return self.average_pedestal_per_chip 

In [60]:
num_chips = 16#Easy Visualisation

In [61]:
benchmark_data = simData(num_chips,27).chip_average()

benchmark_probability = softmax(benchmark_data)


kl_div_data=pd.DataFrame(index=np.arange(1,num_chips+1))

kl_div_data['BenchmarkScore'] = kl_div(benchmark_probability,benchmark_probability)

In [62]:
#KL-Divergence for Similar Distribution

test_1_laplacian = simData(num_chips,55).chip_average()

test_1_laplacian_probability = softmax(test_1_laplacian)

kl_div_data['Similar Distributions - [Test1]']  = np.round(kl_div(benchmark_probability,test_1_laplacian_probability),3)

In [63]:
#KL-Divergence for Data with Random Outliers

test_2_outliers = benchmark_data.copy()

test_2_outliers[np.random.randint(num_chips,size=12)]=test_2_outliers[np.random.randint(num_chips,size=12)]+10

test_2_outliers_probability = softmax(test_2_outliers)

kl_div_data['Data with Random Outliers - [Test2]']  = np.round(kl_div(benchmark_probability,test_2_outliers_probability),3)

In [64]:
#KL-Divergence for Data with Large Outliers

test_3_large_outliers = benchmark_data.copy()

test_3_large_outliers[np.random.randint(num_chips,size=12)]=test_3_large_outliers[np.random.randint(num_chips,size=12)]+20

test_3_large_outliers_probability = softmax(test_3_large_outliers)

kl_div_data['Data with Large Outliers - [Test3]']  = np.round(kl_div(benchmark_probability,test_3_large_outliers_probability),3)

In [65]:
#KL-Divergence for Data with Zero Value Distribution

test_4_zeros = np.zeros(num_chips)

test_4_zeros_probability = softmax(test_4_zeros)

kl_div_data['Data with Zero Values - [Test4]']  = np.round(kl_div(benchmark_probability,test_4_zeros_probability),3)

In [77]:
#KL-Divergence for Data with Back Ground Noise

test_5_w_noise = benchmark_data.copy()

test_5_w_noise=test_5_w_noise + np.random.normal(0,1,num_chips)

test_5_w_noise_probability = softmax(test_5_w_noise)

kl_div_data['Data with Background Noise - [Test5]']  = np.round(kl_div(benchmark_probability,test_5_w_noise_probability),3)

In [81]:
#KL-Divergence for Data with Systematic Shift

test_6_w_syst_shift = benchmark_data.copy()

test_6_w_syst_shift= 2 * test_6_w_syst_shift

test_6_w_syst_shift_probability = softmax(test_6_w_syst_shift)

kl_div_data['Data with Systematic Shift - [Test6]']  = np.round(kl_div(benchmark_probability,test_6_w_syst_shift_probability),3)

In [84]:
kl_div_data.style.applymap(colorCodes)

,BenchmarkScore,Similar Distributions - [Test1],Data with Random Outliers - [Test2],Data with Large Outliers - [Test3],Data with Zero Values - [Test4],Data with Background Noise - [Test5],Data with Systematic Shift - [Test6]
1,0.000000,0.001000,0.628000,0.018000,0.001000,0.005000,0.001000
2,0.000000,0.005000,0.000000,0.005000,0.001000,0.092000,0.001000
3,0.000000,0.000000,0.015000,1.075000,0.000000,0.139000,0.000000
4,0.000000,0.003000,0.419000,0.017000,0.002000,0.003000,0.002000
5,0.000000,0.000000,0.001000,1.048000,0.000000,0.029000,0.000000
6,0.000000,0.001000,0.006000,0.015000,0.000000,0.043000,0.000000
7,0.000000,0.003000,0.596000,0.008000,0.000000,0.025000,0.000000
8,0.000000,0.003000,0.006000,0.012000,0.000000,0.000000,0.001000
9,0.000000,0.000000,0.502000,0.014000,0.000000,0.057000,0.000000
10,0.000000,0.001000,0.007000,1.094000,0.000000,0.009000,0.000000
